## TODO

TEST BASIC PLUGINS HERE >> MOVE TO .py

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
search_endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
search_key = os.environ["AZURE_SEARCH_ADMIN_KEY"]

In [20]:
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI

from langchain.vectorstores.azuresearch import AzureSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import textwrap

from langchain_core.prompts import ChatPromptTemplate

AzureSearch

In [16]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    chunk_size=1 
)
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=search_endpoint,
    azure_search_key=search_key,
    index_name="boardai03",
    embedding_function=embeddings.embed_query,
)

llm = AzureChatOpenAI(
    deployment_name="gpt-4",
    api_key=azure_openai_key,
    azure_endpoint=azure_openai_endpoint,
    api_version="2023-09-01-preview",
    # temperature=0
)



In [19]:
retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 5})

prompt = hub.pull("rlm/rag-prompt")
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
{"context": retriever | format_docs, "question": RunnablePassthrough()}
| prompt
| llm
| StrOutputParser()
)

query = "Who should be involved in the creation of a board paper?"
res = rag_chain.invoke(query)

print("\n".join(textwrap.wrap(res, width = 140)))

The creation of a board paper should involve the NEC, who write the paper, and the Secretary, who based on the approved agenda, emails the
NEC and later circulates and distributes the paper to board members.


### people_db plugin

In [30]:
retriever = vector_store.as_retriever(search_key="hybrid", search_kwargs={"k": 5})


prompt = """
based on the given question's position or team, give a result all the names, emails of who’s in that position or in the team.

Example:
'Find out who are the board members and NEC'

Board Members:
-	Mike CEO - mike@mycompany.com - CEO
-	Olivia Johnson - olivia@mycompany.com - CFO
-	Alex Rodriguez - alex@mycompany.com - COO
NEC:
-	Emily Smith - emily@mycompany.com - CTO
-	Victoria Chen - victoria@mycompany.com - CMO
-	Michael Turner - michael@mycompany.com - CIO


\nTask: {task}
\nContext: {context}


\nAnswer:""" 

prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])

task = "who are the board members?"

rag_chain = (
{"context": retriever | format_docs, "task": RunnablePassthrough()}
| prompt_template
| llm
| StrOutputParser()
)

res = rag_chain.invoke(task)

print("\n".join(textwrap.wrap(res, width = 140)))

The board members are:  - Mike CEO - mike@mycompany.com - CEO - Olivia Johnson - olivia@mycompany.com - CFO - Alex Rodriguez -
alex@mycompany.com - COO
